In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split,cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression,Lasso,Ridge,BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from IPython.display import Image
from pandas_profiling import ProfileReport


In [2]:
final_test = pd.read_csv('data/test.csv')
train_init = pd.read_csv('data/train.csv')
train_init = train_init.drop(columns=["Id"])
# train_init.columns
# train_init.head()

In [3]:

labels = train_init["SalePrice"]
features = train_init.drop(columns=["SalePrice"])

# Set up training and testing data
x_train,x_test,y_train,y_test=train_test_split(features, labels, test_size=0.2, random_state=7)

In [4]:
df = features
# Create a boolean mask indicating the presence of missing values in each column
mask = df.isna()
# Use the sum method to count the number of missing values in each column
na_percent = mask.mean()
sorted_na_percent = na_percent.sort_values(ascending=False)

# Print the column names and percentage of missing values for columns with missing values
# print(sorted_na_percent[sorted_na_percent > 0])

In [9]:
df.fillna('No', inplace=True)
df.head(10).style.background_gradient(cmap = "viridis")
df.describe().transpose().style.background_gradient(cmap = "magma")
# df.info

,count,mean,std,min,25%,50%,75%,max
MSSubClass,1460.000000,56.897260,42.300571,20.000000,20.000000,50.000000,70.000000,190.000000
LotArea,1460.000000,10516.828082,9981.264932,1300.000000,7553.500000,9478.500000,11601.500000,215245.000000
OverallQual,1460.000000,6.099315,1.382997,1.000000,5.000000,6.000000,7.000000,10.000000
OverallCond,1460.000000,5.575342,1.112799,1.000000,5.000000,5.000000,6.000000,9.000000
YearBuilt,1460.000000,1971.267808,30.202904,1872.000000,1954.000000,1973.000000,2000.000000,2010.000000
YearRemodAdd,1460.000000,1984.865753,20.645407,1950.000000,1967.000000,1994.000000,2004.000000,2010.000000
BsmtFinSF1,1460.000000,443.639726,456.098091,0.000000,0.000000,383.500000,712.250000,5644.000000
BsmtFinSF2,1460.000000,46.549315,161.319273,0.000000,0.000000,0.000000,0.000000,1474.000000
BsmtUnfSF,1460.000000,567.240411,441.866955,0.000000,223.000000,477.500000,808.000000,2336.000000
TotalBsmtSF,1460.000000,1057.429452,438.705324,0.000000,795.750000,991.500000,1298.250000,6110.000000


In [10]:
df.describe().transpose().style.background_gradient(cmap = "magma")

,count,mean,std,min,25%,50%,75%,max
MSSubClass,1460.000000,56.897260,42.300571,20.000000,20.000000,50.000000,70.000000,190.000000
LotArea,1460.000000,10516.828082,9981.264932,1300.000000,7553.500000,9478.500000,11601.500000,215245.000000
OverallQual,1460.000000,6.099315,1.382997,1.000000,5.000000,6.000000,7.000000,10.000000
OverallCond,1460.000000,5.575342,1.112799,1.000000,5.000000,5.000000,6.000000,9.000000
YearBuilt,1460.000000,1971.267808,30.202904,1872.000000,1954.000000,1973.000000,2000.000000,2010.000000
YearRemodAdd,1460.000000,1984.865753,20.645407,1950.000000,1967.000000,1994.000000,2004.000000,2010.000000
BsmtFinSF1,1460.000000,443.639726,456.098091,0.000000,0.000000,383.500000,712.250000,5644.000000
BsmtFinSF2,1460.000000,46.549315,161.319273,0.000000,0.000000,0.000000,0.000000,1474.000000
BsmtUnfSF,1460.000000,567.240411,441.866955,0.000000,223.000000,477.500000,808.000000,2336.000000
TotalBsmtSF,1460.000000,1057.429452,438.705324,0.000000,795.750000,991.500000,1298.250000,6110.000000


In [5]:
# var_num = ["SalePrice", "OverallQual", "GrLivArea", "GarageCars", "TotalBsmtSF", "FullBath", "YearBuilt"]
# sns.pairplot(df[var_num]);

profile = ProfileReport(df, title="Pandas Profiling Report")
profile.to_file("profile.html")




Summarize dataset:  93%|█████████▎| 80/86 [00:01<00:00, 12.21it/s, Calculate auto correlation]     c:\Users\Ruby\anaconda3\lib\site-packages\pandas_profiling\model\correlations.py:67: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  warnings.warn(
Export report to file: 100%|██████████| 1/1 [00:00<00:00,  3.75it/s]
